In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from tqdm import tqdm
import uuid

from datasets import Dataset

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
url = "https://www.kinomania.ru/film/441560/script/262"
response = requests.get(url=url)
soup = BeautifulSoup(response.text)
movie_description = soup.find("div", class_="scripts-info-caption")
print(movie_description.text)
movie_script = soup.find("div", class_="row-text")
print(movie_script.text)
title = soup.find("h1", class_="pagetitle").text
print(title)

'Чужой против Хищника'

In [4]:
class KinomaniaScriptsParser:
    def __init__(self) -> None:
        self.urls = []
        self.dataset = []

    def parse_scripts(self):
        self.urls = self.parse_scripts_links()

        for url in tqdm(self.urls):
            response = requests.get(url=url)
            soup = BeautifulSoup(response.text)

            movie_description = soup.find("div", class_="scripts-info-caption").text
            movie_script = soup.find("div", class_="row-text").text
            title = soup.find("h1", class_="pagetitle").text

            self.dataset.append(
                {
                    "title": title,
                    "movie_description": movie_description,
                    "movie_script": movie_script,
                }
            )

    def parse_scripts_links(self):
        url = "https://www.kinomania.ru/scripts"
        response = requests.get(url=url)
        soup = BeautifulSoup(response.text)
        all_scripts = soup.find_all("div", class_="list-tile-item")
        all_links = []
        for item in all_scripts:
            script_href = item.find("a")["href"]
            script_href = f"https://www.kinomania.ru/{script_href}"
            all_links.append(script_href)

        return all_links


parser = KinomaniaScriptsParser()
parser.parse_scripts()

100%|██████████| 27/27 [00:14<00:00,  1.92it/s]


In [11]:
url = "https://www.kinomania.ru/scripts"
response = requests.get(url=url)
soup = BeautifulSoup(response.text)
all_scripts = soup.find_all("div", class_="list-tile-item")

In [15]:
script_href = all_scripts[0].find("a")["href"]
script_href = f"https://www.kinomania.ru/{script_href}"
script_href

'https://www.kinomania.ru//film/401988/script/268'

### convert to dataset


In [43]:
def remove_new_lines(string):
    string = re.sub(r"[\n\t\r]{3,}", "\n\n", string)
    string = re.sub(r"[ ]{2,}", " ", string)
    return string


dataset = parser.dataset

# text = remove_new_lines(dataset[1]['movie_script'])
for i in range(len(dataset)):
    dataset[i]["movie_script"] = remove_new_lines(dataset[i]["movie_script"])

In [5]:
pd.DataFrame(data=dataset).to_csv(
    "./verbalist/datasets/kinomania_scripts/kinomania_scripts.csv",
    index=False,
)

In [10]:
dataset = pd.read_csv("./verbalist/datasets/kinomania_scripts/kinomania_scripts.csv")
dataset = dataset.to_dict('records')
dataset = {item['title']: item for item in dataset}

dataset2 = pd.read_csv("./verbalist/datasets/kinomania_scripts/kinomania_scripts_translated_summary.csv")
dataset2 = dataset2.to_dict('records')
for item in dataset2:
    dataset[item['title']]['movie_description'] = item['movie_description']

dataset = list(dataset.values())

In [14]:
dataset = Dataset.from_list(dataset)
dataset

Dataset({
    features: ['movie_script', 'movie_description', 'title'],
    num_rows: 27
})

In [15]:
dataset.push_to_hub("dim/kinomania_scripts")

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:02<00:00,  2.37s/it]
